In [2]:
pip install mlxtend

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas
from mlxtend.preprocessing import TransactionEncoder

In [5]:
#read data
dinosaurList = pandas.read_csv('dinosaur.csv')

In [6]:
#print data
display(dinosaurList)

,Name,Period,Diet,Country
0,Aardonyx,Jurassic,herbivore,South Africa
1,Abelisaurus,Cretaceous,carnivore,South America
2,Abrictosaurus,Jurassic,herbivore,South Africa
3,Abrosaurus,Jurassic,herbivore,China
4,Abydosaurus,Cretaceous,herbivore,North America
...,...,...,...,...
1149,Zizhongosaurus,Jurassic,herbivore,China
1150,Zuniceratops,Cretaceous,herbivore,North America
1151,Zuolong,Jurassic,carnivore,China
1152,Zupaysaurus,Triassic,carnivore,South America


In [7]:
"""
#Itemset generation for finding association between the period and the diet of the dinosaurs.
"""

# Normalize, strip and lower case the values of Period and Diet columns

dinosaurList["Period"] = dinosaurList["Period"].str.normalize('NFKD')
dinosaurList["Diet"] = dinosaurList["Diet"].str.normalize('NFKD')
dinosaurList["Period"] = dinosaurList["Period"].str.strip()
dinosaurList["Diet"] = dinosaurList["Diet"].str.strip()
dinosaurList["Period"] = dinosaurList["Period"].str.replace('(','').replace(')','')
dinosaurList["Diet"] = dinosaurList["Diet"].str.replace('(','')
dinosaurList["Diet"] = dinosaurList["Diet"].str.replace(')','')
dinosaurList["Diet"] = dinosaurList["Diet"].str.replace('?','')
dinosaurList["Period"] = dinosaurList["Period"].str.lower()
dinosaurList["Diet"] = dinosaurList["Diet"].str.lower() 

# Creating a new dataframe with data from Period and Diet columns
dataa = [list(dList) for dList in zip(dinosaurList["Period"], dinosaurList["Diet"])]

C:\Users\Puneet\AppData\Local\Temp\ipykernel_30624\3504220584.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dinosaurList["Period"] = dinosaurList["Period"].str.replace('(','').replace(')','')
C:\Users\Puneet\AppData\Local\Temp\ipykernel_30624\3504220584.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dinosaurList["Diet"] = dinosaurList["Diet"].str.replace('(','')
C:\Users\Puneet\AppData\Local\Temp\ipykernel_30624\3504220584.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dinosaurList["Diet"] = dinosaurLi

In [8]:
#Using transaction encoder to transform the dataframe
trnxnEncdr = TransactionEncoder()
trnxnEncdrArry = trnxnEncdr.fit(dataa).transform(dataa)
encodedDF = pandas.DataFrame(trnxnEncdrArry, columns=trnxnEncdr.columns_)
encodedDF

,,carnivore,carnivore/insectivore,carnivore/omnivore,cretaceous,early cretaceous,early jurassic,early-late cretaceous,herbivore,herbivore/omnivore,...,late cretaceous,late triassic,middle jurassic,omnivore,omnivorous,triassic,triassic or jurassic,triassic/jurassic,unknown,unknown)
0,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,False,True,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,True,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1150,False,False,False,False,True,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1151,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1152,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False


In [9]:
from mlxtend.frequent_patterns import apriori

#Minimum support in this experiment = 10%
#Minimum confidence = 10%
# Generating frequent itemsets
frqIt = apriori(encodedDF, min_support=0.1, use_colnames=True)
frqIt['length'] = frqIt['itemsets'].apply(lambda x: len(x))
frqIt

,support,itemsets,length
0,0.303293,(carnivore),1
1,0.662045,(cretaceous),1
2,0.624783,(herbivore),1
3,0.229636,(jurassic),1
4,0.188042,"(carnivore, cretaceous)",2
5,0.419411,"(cretaceous, herbivore)",2
6,0.149913,"(jurassic, herbivore)",2


In [10]:
# Filtering frequent itemsets of length 2 and support >= 10%
frqIt[ (frqIt['length'] == 2) & (frqIt['support'] >= 0.1) ]

,support,itemsets,length
4,0.188042,"(carnivore, cretaceous)",2
5,0.419411,"(cretaceous, herbivore)",2
6,0.149913,"(jurassic, herbivore)",2


In [11]:
"""
#Rule Generation using specified confidence value
"""
from mlxtend.frequent_patterns import association_rules

assRules = association_rules(frqIt, metric="confidence", min_threshold=0.1)

assRules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(carnivore),(cretaceous),0.303293,0.662045,0.188042,0.620000,0.936492,-0.012752,0.889355
1,(cretaceous),(carnivore),0.662045,0.303293,0.188042,0.284031,0.936492,-0.012752,0.973097
2,(cretaceous),(herbivore),0.662045,0.624783,0.419411,0.633508,1.013964,0.005776,1.023805
3,(herbivore),(cretaceous),0.624783,0.662045,0.419411,0.671290,1.013964,0.005776,1.028125
4,(jurassic),(herbivore),0.229636,0.624783,0.149913,0.652830,1.044890,0.006441,1.080787
5,(herbivore),(jurassic),0.624783,0.229636,0.149913,0.239945,1.044890,0.006441,1.013563
